In [1]:
import pandas as pd
import re
import os
import email
import dateutil.parser as dateparser
import datetime

In [2]:
'''
(Note for AY)
Current issue with zipper-a: 
?
 -----Original Message-----
From: =09Mack, Iris =20
Sent:=09Wednesday, September 26, 2001 12:37 PM

shows -9114 days reply time. 
'''

'\n(Note for AY)\nCurrent issue with zipper-a: \n?\n -----Original Message-----\nFrom: =09Mack, Iris =20\nSent:=09Wednesday, September 26, 2001 12:37 PM\n\nshows -9114 days reply time. \n'

In [2]:
# Constants
# Tested and verified OK- small: lokey-t, south-s
user = "zipper-a" # NOTE: SOME PEOPLE DO NOT HAVE THE SAME REPLY FORMAT. 
send_folders = ["_sent_mail", "sent", "sent_items"] # do not edit this!

In [3]:
# Get the number of emails in user's folder. 
def get_num_sent_emails(user, foldername):
    n = (len(os.listdir("data/maildir/"+user+"/"+foldername)))
    return n

In [4]:
def get_user_email_id(user):
    send_folder = send_folders[0] # doesn't actually matter which folder we use.
    # get the "from" field of one of the sent emails
    sentfilename = os.listdir("data/maildir/" + user + "/" + send_folder)[0]
    fullfilename = "data/maildir/" + user + "/" + send_folder + "/" + sentfilename
    with open(fullfilename, "r") as f:
        for line in f:
            if line.startswith("From:"):
                return line.split(": ")[1].strip()
            

# print("Email ID for user {}: ".format(user), get_user_email_id(user))

In [5]:
## Make the dataframe, to be filled with data. 
df = pd.DataFrame(columns=["uid", "sender", "recipient", "sendtime", "replytime", \
                           "replyspeed", "messagecontent", "foldername"])
df.set_index("uid", inplace=True)
# Indices of the dataframe are messageIDs. (because properness.)

In [6]:
# Choose which sent folder to use.
sent_folder_name = ""
for folder in send_folders:
    if os.path.isdir("data/maildir/"+user+"/"+folder):
        if sent_folder_name == "":
            sent_folder_name = folder
        if get_num_sent_emails(user, folder) > get_num_sent_emails(user, sent_folder_name):
            sent_folder_name = folder


print("Using folder: ", sent_folder_name)


Using folder:  sent_items


In [7]:
def isReply(content):    
    # REGARDLESS of whether we have a fwd or a reply, we are looking for a block 
    # that looks like this:
    
    # SENDER on DATE 05/16/2001 05:30:21 PM
    # To: SOMEONE
    # cc: ??
    # Subject: xxx
    time_regex = "\d+:\d\d(|:\d\d)( |)(AM|PM)"
    date_slash_regex = "\d\d/\d\d/20\d\d " + time_regex
    whitespace_regex = r"[\s(=20)(=09)]+"
    a = re.search(date_slash_regex + whitespace_regex + r"To:", content)
    if a is not None:
        just_date = re.search(date_slash_regex, a.group()).group()
        return just_date
    
    # Format:
    # -----Original Message-----
    # From: whatever
    # Sent: Friday, October 05, 2001 3:07 PM
    day_regex = r"(\w+)day, \w+ \d\d, 20\d\d " + time_regex
    a = re.search(day_regex, content)
    if a is not None:
        return a.group()
        


    else:
        return -1



In [8]:
# Iterate through the sent emails and add to dataframe. 
filenames = os.listdir("data/maildir/"+user+"/"+sent_folder_name)
for file in filenames:
    full_filename = "data/maildir/"+user+"/"+sent_folder_name+"/"+file
    f = open(full_filename, "r")

    # Process the message
    message = email.message_from_file(f)
    f.close()

    # Get the list of recipients.
    # print(message.items())
    if message["To"]:
        recipient_list = message["To"].split(", ")
    if message["Cc"]:
        recipient_list += message["Cc"].split(", ")
    if message["Bcc"]:
        recipient_list += message["Bcc"].split(", ")
    # if message["X-To"]:
    #     recipient_list += message["X-To"].split(", ")
    # if message["X-cc"]:
    #     recipient_list += message["X-cc"].split(", ")
    # if message["X-bcc"]:
        recipient_list += message["X-bcc"].split(", ")
    recipient_list = list(set([r.strip() for r in recipient_list])) # remove duplicates
    if len(recipient_list) == 0:
        continue # Email not sent to anyone?

    # Get the time the email was sent.
    sendtime = dateparser.parse(message["Date"])
    # print("send time", sendtime.astimezone(datetime.timezone.utc))

    # Calculate the reply speed:
    prev_email_time = isReply(content=message.get_payload())
    if prev_email_time != -1:
        # print("reply time", prev_email_time)
        prev_email_time = dateparser.parse(prev_email_time + "+0000")
        # print("prev_email_time", prev_email_time.astimezone(datetime.timezone.utc))
        replyspeed = sendtime - prev_email_time
        # print("replyspeed", replyspeed)
    else:
        replyspeed = -1

    # print(message.get_payload()[:300])
    # print("---")

    # Make dataframe row(s):
    for recipient in recipient_list:
        row = {
            "uid": message["Message-ID"]+recipient,
            "sender": message["From"],
            "recipient": recipient,
            "sendtime": sendtime,
            "replytime": prev_email_time, 
            "replyspeed": replyspeed,
            "messagecontent": message.get_payload(),
            "foldername": ""
        }
        id = message["Message-ID"]+recipient,
        entry = pd.DataFrame([row])
        df = pd.concat([df, entry], ignore_index=True)

df

    


,sender,recipient,sendtime,replytime,replyspeed,messagecontent,foldername,uid
0,andy.zipper@enron.com,john_bourget@bankone.com,2002-01-07 13:21:50-08:00,-1,-1,"George Bush, minister of the double negative: ...",,<23245147.1075842017323.JavaMail.evans@thyme>j...
1,andy.zipper@enron.com,steve.wang@enron.com,2001-10-02 05:44:29-07:00,-1,-1,Month\tVol\t\nNov-01\t0.7700\t\nDec-01\t0.6675...,,<31662779.1075851549023.JavaMail.evans@thyme>s...
2,andy.zipper@enron.com,michael.bridges@enron.com,2001-06-06 12:53:34-07:00,-1,-1,"What is, exactly, the state of play here ? I n...",,<14257679.1075845424721.JavaMail.evans@thyme>m...
3,andy.zipper@enron.com,kal.shah@enron.com,2001-05-30 10:15:13-07:00,-1,-1,CAN YOU SEND David Lebow in IR a copy of the a...,,<33402920.1075845423137.JavaMail.evans@thyme>k...
4,andy.zipper@enron.com,michael.guadarrama@enron.com,2001-06-05 12:41:30-07:00,-1,-1,Why wouldn't we use the deal bench auction pla...,,<15663581.1075845424335.JavaMail.evans@thyme>m...
...,...,...,...,...,...,...,...,...
434,andy.zipper@enron.com,daniel.diamond@enron.com,2001-06-04 13:26:32-07:00,-1,-1,Let's table this for now. Too much going on.\n...,,<30500266.1075845424177.JavaMail.evans@thyme>d...
435,andy.zipper@enron.com,greg.piper@enron.com,2001-06-04 07:41:33-07:00,2001-06-04 08:55:00+00:00,0 days 05:46:33,\nThis is where we are with banks for fx produ...,,<33433088.1075845423768.JavaMail.evans@thyme>g...
436,andy.zipper@enron.com,bob.shults@enron.com,2001-05-24 14:16:41-07:00,2001-05-23 10:45:00+00:00,1 days 10:31:41,\nPlease call Joe Robertson of enron Metals in...,,<12146514.1075845422622.JavaMail.evans@thyme>b...
437,andy.zipper@enron.com,ina.rangel@enron.com,2001-10-15 08:54:19-07:00,-1,-1,Okay.,,<19367478.1075851548126.JavaMail.evans@thyme>i...


In [9]:
# SAVE AS CSV
!mkdir -p processed_data
df.to_csv("processed_data/"+user+"_sent.csv")